In [5]:
import math
from enum import Enum
import numpy as np
import pandas as pd
from collections import namedtuple, defaultdict
from typing import List, Tuple

In [6]:
NUM_ROWS = 10
NUM_COLS = 10

Pos = namedtuple('Pos', ['r', 'c'])

class NodeState(Enum):
    VACANT = 0
    WALL = 1
    ORIGIN = 'O'
    GOAL = 'G'
    VISITED = 4
    PATH = 'X'

    def __repr__(self):
        return str(self.value)

class Node:
    def __init__(self, pos: Pos, state: NodeState):
        self.state = state
        self.pos = pos
        self.heuristic = None
        self.predecessor = None

    def __repr__(self):
        return str(self.state.value)
        # return f'({str(self.pos.r).zfill(3)}, {str(self.pos.c).zfill(3)})'
        # return f'{self.state.value} - ({self.pos.r}, {self.pos.c})'

In [7]:
def get_neighbours(map_arr: np.array, node: Node) -> List[Tuple]:
    '''
    Given a row and column coordinate, returns a list of neighbours' coordinates.
    '''

    r, c = node.pos.r, node.pos.c
    neighbours = []
    if r > 0:
        neighbours.append(map_arr[r - 1][c])
    if r < NUM_ROWS - 1:
        neighbours.append(map_arr[r + 1][c])
    if c > 0:
        neighbours.append(map_arr[r][c - 1])
    if c < NUM_COLS - 1:
        neighbours.append(map_arr[r][c + 1])
    return neighbours

In [8]:
def a_star(map_arr: np.array, start_node: Node, end_node: Node) -> bool:
    '''
    Finds a path from origin to goal. Currently, squares on a grid are not assigned weights. Weights between nodes are all by default to 1.
    Heuristic is euclidean distance, by assuming lattice nodes exist in the 4th quadrant of an axis. This is why the y co-ords are negated,
    although I'm not sure it makes much of a difference. The heuristic is only considered when choosing which node to select out of a node's
    neighbours. We do not add it to the distance when updating distance from origin node.
    '''

    dist = defaultdict(lambda: float('inf')) # Initially, distance to every other node is infinite. Replaced with tuple of 2 values which indicate a particular node (r, c)
    dist[start_node] = 0 # Distance from start node to start node is 0
    node = start_node
    path_trace = []

    # Calculating the initial heuristic values using Euclidean distances
    x1, y1 = end_node.pos.r, -end_node.pos.c
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS):
            x2, y2 = r, -c
            euclid_distance = math.sqrt(((x2 - x1) ** 2) + ((y2 - y1) ** 2))
            map_arr[r][c].heuristic = euclid_distance

    while True:
        if node.state != NodeState.ORIGIN:
            node.state = NodeState.VISITED
        neighbours = get_neighbours(map_arr, node)
        unvisited_neighbours = list(filter(lambda n: n.state not in [NodeState.VISITED, NodeState.ORIGIN, NodeState.WALL], neighbours))
        for neighbour in unvisited_neighbours:
            if dist[node] + 1 < dist[neighbour]:  # If current distance to a node is smaller than any previous possible distance, update it. Every node is only 1 node away from other nodes due to the fact that this is a nxn grid with no weights.
                dist[neighbour] = dist[node] + 1
                neighbour.predecessor = node
            if neighbour == end_node:
                print('Path found!')
                n = neighbour
                while n.predecessor:
                    if n.state not in [NodeState.ORIGIN, NodeState.GOAL]:
                        n.state = NodeState.PATH
                    n = n.predecessor
                return True
        # "Priority Queue"
        smallest_path_dist, smallest_path_node = float('inf'), None
        for node in dist.keys():
            # print(dist[node], node.heuristic, node.state)
            if  node.state not in [NodeState.VISITED, NodeState.WALL, NodeState.ORIGIN] and (dist[node] + node.heuristic) < smallest_path_dist:
                smallest_path_dist = dist[node] + node.heuristic
                smallest_path_node = node
        node = smallest_path_node
        if not node:
            print('No path found')
            break

In [9]:
def get_random_map_array():
    '''
    Returns a 2D numpy array with randomized vacant and wall nodes. To change the
    ratio of walls/vacant nodes, alter the parameter p in the call to np.random.choice.
    '''
    map_array = []
    for r in range(NUM_ROWS):
        new_row = []
        for c in range(NUM_COLS):
            random_state = np.random.choice([NodeState.VACANT, NodeState.WALL], p=[0.8, 0.2])
            new_random_node = Node(Pos(r, c), random_state)
            new_row.append(new_random_node)
        map_array.append(new_row)
    map_array[NUM_ROWS - 1][NUM_COLS - 1] = Node(Pos(NUM_ROWS - 1, NUM_COLS - 1), NodeState.ORIGIN)
    map_array[0][0] = Node(Pos(0, 0), NodeState.GOAL)
    return np.array(map_array)

map_arr = get_random_map_array()
start_node = map_arr[NUM_ROWS - 1][NUM_COLS - 1] # (r, c) - bottom right of grid
end_node = map_arr[0][0] # - top left of grid
a_star(map_arr, start_node, end_node)

print(map_arr)

Path found!
[[G X 4 4 4 1 4 0 4 0]
 [0 X X 4 4 4 4 1 4 0]
 [4 4 X 4 4 4 4 4 4 1]
 [0 1 X 1 1 4 4 4 4 4]
 [4 4 X 1 4 1 1 4 4 1]
 [4 4 X X X X X 4 4 4]
 [0 1 1 4 4 4 X 1 1 4]
 [0 1 1 4 4 4 X X X X]
 [0 0 1 4 4 1 4 4 1 X]
 [0 0 1 4 4 4 4 4 4 O]]
